<a href="https://colab.research.google.com/github/YooYeonjin2/-/blob/main/n222-random-forests/n222a-random-forests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 2 / SPRINT 2 / NOTE 2*

# 📝 Assignment
---

# 랜덤포레스트(Random Forests)

### 1) 캐글 대회를 이어서 진행합니다. EDA, 데이터 전처리 부분을 업데이트 하세요.
- EDA는 항상 완벽할 수 없지요 한 주간 계속 같은 데이터로 과제를 진행하므로 부족한 부분을 추가하거나 논하세요.
- (지금은 feature engineering에 너무 시간을 들이지 마세요!)
- Ordinal Encoding을 적용해 보세요.
- **(Urclass Quiz) 다음 특성들 중에 순서를 고려한 Ordinal Encoding이 필요해 보이는 특성을 고르세요.**
    1. opinion_h1n1_vacc_effective
    2. state
    3. marital
    4. employment_occupation


In [ ]:
!pip install category_encoders

In [2]:
import pandas as pd
target = 'vacc_h1n1_f'
train = pd.merge(pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train.csv'),
                 pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train_labels.csv')[target], left_index=True, right_index=True)
test = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/test.csv')
sample_submission = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/submission.csv')

In [4]:
train['employment_occupation'].value_counts()

Management Occupations                                       2188
Office and Administrative Support Occupations                1931
Education, Training, and Library Occupations                 1609
Healthcare Practitioners and Technical Occupations           1535
Sales and Related Occupations                                1404
Business and Financial Operations Occupations                 946
Construction and Extraction Occupations                       672
Production Occupations                                        614
Transportation and Material Moving Occupations                612
Computer and Mathematical Occupations                         589
Food Preparation and Serving Related Occupations              523
Arts, Design, Entertainment, Sports and Media Occupations     457
Architecture and Engineering Occupations                      447
Community and Social Services Occupations                     435
Personal Care and Service Occupations                         429
Building a

In [5]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, train_size=0.80, test_size=0.20, stratify = train[target], random_state=2)
train.shape, val.shape, test.shape

((33723, 39), (8431, 39), (28104, 38))

In [6]:
def engineer(df):
    """특성을 엔지니어링 하는 함수입니다."""
    
    # 높은 카디널리티를 가지는 특성을 제거합니다.
    selected_cols = df.select_dtypes(include=['number', 'object'])
    colnames = selected_cols.columns.tolist()
    labels = selected_cols.nunique()
    
    selected_features = labels[labels <= 30].index.tolist()
    df = df[selected_features]
    
    # 새로운 특성을 생성합니다.
    behaviorals = [col for col in df.columns if 'behavioral' in col] 
    df['behaviorals'] = df[behaviorals].sum(axis=1)
    
    
    dels = [col for col in df.columns if ('employment' in col or 'seas' in col)]
    df.drop(columns=dels, inplace=True)
        
    return df


train = engineer(train)
val = engineer(val)
test = engineer(test)

In [7]:
features = train.drop(columns=[target]).columns

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [8]:
from category_encoders import OrdinalEncoder
encoder = OrdinalEncoder()
X_train_encoded = encoder.fit(X_train)

### 2) 랜덤포레스트 모델을 적용한 후의 결과를 캐글에 제출하세요.
- 랜덤포레스트를 적용하고 성능이 오히려 떨어졌을 수도 있습니다! 만약 그렇다면 이유를 본인 논리로 분석해 보세요.
- **(Urclass Quiz) 캐글 Leaderboard의 본인 Score를 제출하세요.**

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer 
from sklearn.pipeline import make_pipeline
pipe_ord = make_pipeline(
    OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(random_state=10, n_jobs=-1, oob_score=True)
)

pipe_ord.fit(X_train, y_train)
print('검증 정확도: ', pipe_ord.score(X_val, y_val))

검증 정확도:  0.8239829201755426


In [18]:
y_test_pred = pipe_ord.predict(X_test)
submission = sample_submission
submission[target] = y_test_pred.copy()
submission.to_csv('submission_.csv',index=False)

## 🔥 도전과제(Github - Discussion)

### 3) 수업에 사용하지 않은 다른 종류의 [category_encoders](http://contrib.scikit-learn.org/category_encoders/)을 2개 이상 사용해 결과를 공유해 보시고, 다음 질문에 대해 서로 논의해 보세요. 

- 사용하신 encoder는 각각 어떤 장단점을 갖고 있으며, 어떤 상황에서 사용하면 좋을까요? 

### 4) 왜 트리모델에서는 ordinal encoding을 주로 사용하며 (one-hot encoding대신), 범주형 자료를 ordinal encoding으로 사용해도 되는 이유는 무엇이라고 생각하시는지 논의해 보세요


## 참고자료

- [Random Forests for Complete Beginners](https://victorzhou.com/blog/intro-to-random-forests/)